### If you want to replicate the project below the dependency list.


| Library         | Version      |
| --------------- |-------------:|
| tensorflow      | 2.3.0        |
| keras           | 2.4.3        |
| MTCNN           | 0.1.0        |

In [5]:
import cv2
import numpy as np
import tensorflow as tf
import anvil.media
import anvil.server
from mtcnn import MTCNN

detector = MTCNN()

img_size = 64

model = tf.keras.models.load_model("yu4u-age-gender.model")

anvil.server.connect('3HZIYGCZ64G5GHMHXHBUYJKY-QE53PUIDL6P654KY')

@anvil.server.callable
def profile_image(file):
  with anvil.media.TempFile(file) as filename:
    img = cv2.imread(filename)
  input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_h, img_w, _ = np.shape(input_img)

  faces = detector.detect_faces(img)

  all_labels = [];

  margin = 0.2
  for face in faces:
      bounding_box = face['box']
      if face['confidence']>.90: 
          x1, y1, x2, y2, w, h =  bounding_box[0], bounding_box[1],bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3],bounding_box[2],bounding_box[3]
          xw1 = max(int(x1 - margin * w), 0)
          yw1 = max(int(y1 - margin * h), 0)
          xw2 = min(int(x2 + margin * w), img_w - 1)
          yw2 = min(int(y2 + margin * h), img_h - 1)

          face_boundary = cv2.resize(img[yw1:yw2+1, xw1:xw2+1],(img_size,img_size)).reshape(-1,64,64,3)

          # predict ages and genders of the detected faces
          results = model.predict(face_boundary)
          predicted_genders = results[0]
          ages = np.arange(0, 101).reshape(101, 1)
          predicted_ages = results[1].dot(ages).flatten()

          pred_gender = "Male" if predicted_genders[0][0] < 0.5 else "Female"
          label_str = "{} of {} years".format(pred_gender,int(predicted_ages[0]))
          all_labels.append(label_str)
          
  return all_labels

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


WARNING - 6 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f66e97d77b8> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
WARNING - 6 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f66e97d77b8> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.func